<a href="https://colab.research.google.com/github/luca-arts/seeingtheimperceptible/blob/main/notebooks/colorGrading/tests/LPTN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Color grading via LPTN

[https://github.com/csjliang/LPTN](https://github.com/csjliang/LPTN)


In [11]:
# we'll link the dataset from next-cloud
!curl https://raw.githubusercontent.com/luca-arts/seeingtheimperceptible/main/notebooks/database_mod.py -o /content/database_mod.py

from database_mod import *

link_nextcloud()

nextcloud = '/content/database'

input_folder, output_folder = create_io(database=nextcloud,topic='colorGrading',library='LPTN')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1793  100  1793    0     0   9588      0 --:--:-- --:--:-- --:--:--  9537
what's the username for nextcloud? colab
what's the password for user colab? ··········
1
Please enter the username to authenticate with server
https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ or hit enter for none.
  Username: Please enter the password to authenticate user colab with server
https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ or hit enter for none.
  Password:  
content of /etc/fstab: https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ /content/database davfs user,rw,auto 0 0


## 2. Clone the Git repository

In [5]:
import os
root_path = '/content/LPTN'
# clone the repository
if not os.path.exists('LPTN'):
  !git clone https://github.com/csjliang/LPTN {root_path}

fatal: destination path '/content/LPTN' already exists and is not an empty directory.


## 3. install dependencies

In [6]:
!pip install -qr {root_path}/requirement.txt

## 5. Pretrained model download

In [8]:
# check first if exist, otherwise create path
pretrained_pth = root_path + '/experiments/pretrained_models/net_g_FiveK_numhigh3.pth'

if not os.path.exists(pretrained_pth):
  !gdown --id 11yuFgHqZe9e4OheJ9YHp9lS-4kQC1Fml \
          -O {pretrained_pth}

Downloading...
From: https://drive.google.com/uc?id=11yuFgHqZe9e4OheJ9YHp9lS-4kQC1Fml
To: /content/LPTN/experiments/pretrained_models/net_g_FiveK_numhigh3.pth
100% 2.48M/2.48M [00:00<00:00, 14.1MB/s]


## 6. install pyYAML and update the YAML file with correct path for pretrained_models

geen idee of we dit nodig hebben, maar heb zelf deze code erin gezet om te checken of het path effectief naar de goeie folder wijst.


In [9]:
!pip install pyYAML

import yaml

# dowload the pre-trained model for colorgrading
 
def read_yaml(LPTN_path):
    """ A function to read YAML file"""
    with open('{}/options/test/LPTN/test_FiveK.yml'.format(LPTN_path)) as f:
        config = yaml.safe_load(f)
 
    return config

read_yaml(root_path)

{'datasets': {'test': {'dataroot_gt': 'datasets/FiveK/FiveK_test_target.lmdb',
   'dataroot_lq': 'datasets/FiveK/FiveK_test_source.lmdb',
   'io_backend': {'type': 'lmdb'},
   'name': 'FiveK_val',
   'type': 'PairedImageDataset'}},
 'manual_seed': 10,
 'model_type': 'LPTNTestModel',
 'name': 'LPTN_FiveK_480p',
 'network_g': {'nrb_high': 3, 'nrb_low': 5, 'num_high': 3, 'type': 'LPTN'},
 'num_gpu': 1,
 'path': {'pretrain_network_g': 'experiments/pretrained_models/net_g_FiveK_numhigh3.pth',
  'strict_load_g': False},
 'val': {'metrics': {'psnr': {'crop_border': 0,
    'test_y_channel': False,
    'type': 'calculate_psnr'}},
  'save_img': True,
  'suffix': None}}

## 7. dataset downloaden

download dataset

In [12]:
import os
import zipfile

# define folders
output_path_dataset = root_path +'/datasets'
output_path_FiveK = output_path_dataset +'/FiveK'
input_path = nextcloud + '/colorGrading/datasets/FiveK_480p.zip'

# make folders
os.makedirs(output_path_dataset, exist_ok=True)
os.makedirs(output_path_FiveK, exist_ok=True)

#!unzip "/content/drive/path/input_file_name.zip" -d "/content/drive/path/output_folder/"
!unzip -q {input_path} -d {output_path_FiveK}

%ls

^C
codes/     experiments/  options/   requirement.txt
datasets/  Figs/         README.md  scripts/


create lmdb

In [ ]:
%cd /content/LPTN/
!PYTHONPATH="/content/LPTN/:${PYTHONPATH}" python scripts/data_preparation/create_lmdb.py

/usr/bin/python3: can't find '__main__' module in '/content/LPTN'


## 8. Evaluation

yml file test

In [ ]:
!pip install basicsr
!PYTHONPATH="/content/LPTN/:${PYTHONPATH}" CUDA_VISIBLE_DEVICES=0 python codes/test.py -opt options/test/LPTN/test_FiveK.yml

speed testing

In [ ]:
!PYTHONPATH="/content/LPTN/:${PYTHONPATH}" CUDA_VISIBLE_DEVICES=0 python codes/test_speed.py -opt options/test/LPTN/test_speed_FiveK.yml